# mini_project_3.ipynb

Group Members:
- Peter Bollhorn
- Tobias Thormod Birk Nielsen

This notebook presents our solution to Mini Project 3.

In [1]:
import sys
sys.path.append("..")

import pandas as pd
from reader import generic_reader

## Task 1: Data wrangling and exploration

In [2]:
danish_movies = generic_reader.read_csv_file_to_data_frame("../movie_data/danish_movies.csv")
danish_movies

,adult,backdrop_path,budget,homepage,id,imdb_id,origin_country,original_language,original_title,overview,...,video,vote_average,vote_count,genre_ids,production_company_ids,collection_id,cast_person_ids,cast_credit_ids,crew_person_ids,crew_credit_ids
0,False,NaN,0,NaN,195139,tt0290709,['DK'],da,Kørsel med Grønlandske hunde,"Johan Carl Joensen, a Danish colonial manager ...",...,False,5.3,15,[99],[41774],NaN,[2452695],['5dc5d410470ead001391303b'],"[1171313, 1171313]","['52fe4d069251416c91108121', '5dc5d3ef470ead00..."
1,False,NaN,0,NaN,232636,tt0348157,['DK'],da,Svanerne i Sortedamssøen,Swans getting handled as people watch.,...,False,3.0,2,[99],[41774],NaN,[],[],[1171313],['52fe4e07c3a36847f827a04d']
2,False,NaN,0,NaN,195364,tt0346587,['DK'],da,Bech Olsens og Poul Pons' brydekamp,Early wrestling footage.,...,False,4.0,5,[99],[41774],NaN,"[4658231, 4658234]","['661fe77220af77017d3f88a8', '661fe78a6d9fe801...",[1171313],['52fe4d0d9251416c91108ee5']
3,False,NaN,0,NaN,232637,tt1469330,['DK'],da,Badescener fra Skovshoved,Fun at the beach.,...,False,4.5,4,[99],[41774],NaN,[],[],[1171313],['52fe4e07c3a36847f827a057']
4,False,NaN,0,NaN,195305,tt0347357,['DK'],da,De Kongelige skal fotograferes,"Filmed while the Christian IX, King of Denmark...",...,False,4.8,4,[99],[],NaN,[],[],[1171313],['52fe4d0b9251416c91108c3d']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5129,False,/MSP35R6y0ypmf3zU1kyUw1P4It.jpg,0,NaN,1407803,NaN,['DK'],da,Verdensmænd - Bobos surprise,NaN,...,False,10.0,1,[35],[758],NaN,"[1828049, 234058, 1355004, 88356, 2208065]","['676d44614ca2fbbfca614283', '676d44784ca2fbbf...",[],[]
5130,False,/2iVVzCEnX4QMEeyd3e52juYcsH8.jpg,0,NaN,1410638,NaN,['DK'],da,Gud bevare Danmark,NaN,...,False,0.0,0,[],[],NaN,[],[],[],[]
5131,False,/ej5uQVMJTzq8nZBCbf6pUMNF6jl.jpg,0,NaN,1409581,NaN,['DK'],da,Frank Hvam - Nobody,Frank Hvam has lived in New Zealand with his f...,...,False,6.0,1,[35],[758],NaN,[89973],['67724668d8dc22e6c6927586'],"[89973, 5520795, 563909, 5227452, 4263643, 516...","['68650819af3c4b2566414b34', '686508769febd451..."
5132,False,NaN,0,NaN,1412034,NaN,['DK'],da,"Dan Andersen - Nedsat Hørelse, Nedsat Sædkvali...",NaN,...,False,0.0,0,[35],[],NaN,[149860],['677903662b097b15a274ab1e'],[],[]


In [3]:
danish_actors = generic_reader.read_csv_file_to_data_frame("../movie_data/danish_actors.csv")
danish_actors

,actor_id,movie_ids,adult,also_known_as,biography,birthday,deathday,gender,homepage,imdb_id,known_for_department,name,place_of_birth,popularity,profile_path
0,42,"{658817, 456325, 29445, 1354636, 752908, 14460...",False,"['Ларс фон Триер', '拉斯·馮·提爾', 'ラース・フォン・トリアー', ...",Lars von Trier (born Lars Trier; 30 April 1956...,1956-04-30,NaN,2,NaN,nm0001885,Directing,Lars von Trier,"Kongens Lyngby, Danmark",0.9635,/a1zVySI0Yqqf3ORegQ1icMfvhAa.jpg
1,53,{15843},False,"['Rolf Peter Ingvar Storm', 'پیتر استورماره', ...","Peter Stormare (August 27, 1953) was born in A...",1953-08-27,NaN,2,NaN,nm0001780,Acting,Peter Stormare,"Arbrå, Gävleborgs län, Sweden",5.0311,/1rtpuUqBV29jDc1huUhtjGDbEwn.jpg
2,169,{8883},False,[],Hanns Zischler (born 18 June 1947) is a German...,1947-06-18,NaN,2,NaN,nm0957193,Acting,Hanns Zischler,"Nuremberg, Bavaria, Germany",0.2330,/5O4Dum1OuKI4RMB24xjACkL22iA.jpg
3,378,{266285},False,"['Джонатан Прайс', '조너선 프라이스', '乔纳森·普雷斯', 'جان...","Jonathan Pryce, CBE (born 1 June 1947) is a We...",1947-06-01,NaN,2,NaN,nm0000596,Acting,Jonathan Pryce,"Carmel, Flintshire, Wales, UK",2.1896,/zwSv5uXzPTtmitFe39UdqnVwmdL.jpg
4,380,{214137},False,"['Robert DeNiro', ' Robert Anthony De Niro Jr....","Robert Anthony De Niro (born August 17, 1943) ...",1943-08-17,NaN,2,NaN,nm0000134,Acting,Robert De Niro,"Greenwich Village, New York City, New York, USA",3.2940,/cT8htcckIuyI1Lqwt1CvD02ynTh.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15597,5706360,{1154066},False,[],NaN,NaN,NaN,0,NaN,NaN,Acting,Thea Esther Saniel Thomsen,NaN,0.0000,NaN
15598,5706385,{1154066},False,[],NaN,NaN,NaN,0,NaN,NaN,Acting,Morten Jay Jakobsen,NaN,0.0000,NaN
15599,5706390,{1154066},False,[],NaN,NaN,NaN,0,NaN,NaN,Acting,Frederik Rasted,NaN,0.0000,NaN
15600,5706467,{1019835},False,[],NaN,NaN,NaN,0,NaN,NaN,Acting,Jytte Vikkelsøe,NaN,0.0000,NaN


## Task 2: Supervised machine learning: linear regression

## Task 3:  Supervised machine learning: classification

## Task 4: Unsupervised machine learning: clustering